# Sowing Success: How Machine Learning Helps Farmers Select the Best Crops

![](farmer_in_a_field.jpg)

Measuring essential soil metrics such as nitrogen, phosphorous, potassium levels, and pH value is an important aspect of assessing soil condition. However, it can be an expensive and time-consuming process, which can cause farmers to prioritize which metrics to measure based on their budget constraints.

Farmers have various options when it comes to deciding which crop to plant each season. Their primary objective is to maximize the yield of their crops, taking into account different factors. One crucial factor that affects crop growth is the condition of the soil in the field, which can be assessed by measuring basic elements such as nitrogen and potassium levels. Each crop has an ideal soil condition that ensures optimal growth and maximum yield.

A farmer reached out to you as a machine learning expert for assistance in selecting the best crop for his field. They've provided you with a dataset called `soil_measures.csv`, which contains:

- `"N"`: Nitrogen content ratio in the soil
- `"P"`: Phosphorous content ratio in the soil
- `"K"`: Potassium content ratio in the soil
- `"pH"` value of the soil
- `"crop"`: categorical values that contain various crops (target variable).

Each row in this dataset represents various measures of the soil in a particular field. Based on these measurements, the crop specified in the `"crop"` column is the optimal choice for that field.  

In this project, you will build multi-class classification models to predict the type of `"crop"` and identify the single most importance feature for predictive performance.

## Task

***Identify the single feature that has the strongest predictive performance for classifying crop types.***

1. Find the feature in the dataset that produces the best score for predicting `"crop"`.
2. From this information, create a variable called `best_predictive_feature`, which:
	- Should be a dictionary containing the best predictive feature name as a key and the evaluation score (for the metric you chose) as the value.

In [1]:
# All required libraries are imported here for you.
import pandas as pd
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn import metrics
from IPython.display import display
import matplotlib.pyplot as plt

# Load the dataset
crops = pd.read_csv("soil_measures.csv")
display(crops.head())

# Data Exploration
print(f"Number of examples: {crops.shape[0]}", f"Number of features: {crops.shape[1]}", sep='\n')
display(crops['crop'].value_counts().reset_index())
display(crops.isnull().sum().reset_index())

# Features and Predicted values 
XN = crops['N'].values.reshape(-1, 1)
XP = crops['P'].values.reshape(-1, 1)
XK = crops['K'].values.reshape(-1, 1)
Xph = crops['ph'].values.reshape(-1, 1)
y = crops.select_dtypes(['object']).values.reshape(-1)

,N,P,K,ph,crop
0,90,42,43,6.502985,rice
1,85,58,41,7.038096,rice
2,60,55,44,7.840207,rice
3,74,35,40,6.980401,rice
4,78,42,42,7.628473,rice


Number of examples: 2200
Number of features: 5


,crop,count
0,rice,100
1,maize,100
2,chickpea,100
3,kidneybeans,100
4,pigeonpeas,100
5,mothbeans,100
6,mungbean,100
7,blackgram,100
8,lentil,100
9,pomegranate,100


,index,0
0,N,0
1,P,0
2,K,0
3,ph,0
4,crop,0


In [2]:
def logreg_model(feature, target):
	# Split dataset 
	X_train, X_test, y_train, y_test = train_test_split(feature, target, test_size=0.3, random_state=1)

	# Scale numerical variables (2D shape)
	scaler = StandardScaler()
	Xtrain_scaled = scaler.fit_transform(X_train)
	Xtest_scaled = scaler.transform(X_test)

	# Encoding categorical values (1D shape)
	encoder = OneHotEncoder()
	ytrain_encoded = encoder.fit_transform(y_train.reshape(-1, 1))
	ytest_encoded = encoder.fit_transform(y_test.reshape(-1, 1))
	## Convert one-hot encoded vectors back to class labels (returns to 1D shape)
	ytrain = ytrain_encoded.toarray().argmax(axis=1)
	ytest = ytest_encoded.toarray().argmax(axis=1)

	# Logistic regression
	logreg = LogisticRegression()
	logreg.fit(Xtrain_scaled, ytrain)
	y_pred = logreg.predict(Xtest_scaled)
	y_pred_probs = logreg.predict_proba(Xtest_scaled)

	# Convert predicted encoded labels to original class labels
	class_labels = encoder.categories_[0]
	y_pred_names = class_labels[y_pred]
	y_test_names = class_labels[ytest]

	# Compare predicted labels with actual labels
	pred_y = pd.DataFrame(y_pred_names, columns=['predicted'])
	actual_y = pd.DataFrame(y_test_names, columns=['actual'])
	pred_actual_names = pd.concat([pred_y, actual_y], axis=1, join='inner')

	# Calculate accuracy
	feature_performance = round(metrics.f1_score(ytest, y_pred, average='weighted'), 5)	# f1 score
	balanced_accuracy = round(metrics.balanced_accuracy_score(ytest, y_pred), 5)

	return feature_performance, balanced_accuracy

### Guide to evaluating the performance of each feature
You can calculate F1 score, which is the harmonic mean of precision and recall, to evaluate feature performance.

Alternatively, you can use `metrics.balanced_accuracy_score()`.

Scikit-learn's `metrics.f1_score()` function takes the target values, `y_test`, and the predicted values, `y_pred`, in order to calculate the F1 score.
- Set the `f1_score()`'s keyword argument average equal to `"weighted"` when calculating performance for each feature.
- Assign the results of `f1_score()` to a variable called `feature_performance`.

If you created an empty dictionary called `feature_performance` outside of a for loop where you built your models, you can add the feature-performance key-value pairs to the dictionary using the syntax `feature_performance[feature] = feature_importance`.

You can use a `print()` statement with an f-string to output the feature and the performance, for example, `print(f"F1-score for {feature}: {feature_performance}")`

In [3]:
Xfeatures = [XN, XP, XK, Xph]
Xfeature_names = ['N', 'P', 'K', 'ph']
f1_scores = dict()
balanced_accuracy = dict()

for X, Xnames in zip(Xfeatures, Xfeature_names):
	f1_score, balanced_acc = logreg_model(feature=X, target=y)
	f1_scores[f'{Xnames}'] = f1_score
	balanced_accuracy[f'{Xnames}'] = float(balanced_acc)

XF1 = pd.DataFrame(f1_scores, index = [0]).T.reset_index().rename(columns={'index': 'feature', 0: 'f1_score'})
XBAcc = pd.DataFrame(balanced_accuracy, index = [0]).T.reset_index().rename(columns={'index': 'feature', 0: 'balanced_accuracy'})
performance_metrics = pd.merge(XF1, XBAcc, how='inner')

display(performance_metrics)

,feature,f1_score,balanced_accuracy
0,N,0.06874,0.14024
1,P,0.08767,0.19865
2,K,0.12314,0.26519
3,ph,0.04228,0.11088


In [4]:
best_predictive_feature = {'K': f1_scores['K']}
best_predictive_feature

{'K': 0.12314}

Potassium (`K`) appears to be the single most importance feature for predictive performance with the highest F1 score of 0.12 and an accuracy rate of 22.6%.